### **Exportando resultados generados en Google Earth Engine**

En este cuaderno se va a aprender como exportar imágenes o videos a la cuenta de Google Drive del usuario.


#### **1. Preparación del entorno de trabajo**

###### **1.1 Identificación en servicios Google**

Primero se monta la unidad del usuario en **Google Drive**

In [1]:
from google.colab import auth, drive
auth.authenticate_user()

# Montamos la unidad de trabajo de nuestro drive
drive.mount('/content/drive/')

Mounted at /content/drive/


En segundo lugar, es necesario identificarse en el servicio **Google Earth Engine**

In [2]:
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=UcvkQAg-5pA7GmjFRw0v1GDiDxezlVMprMg0zzb8K08&tc=GKZn-tuD87nPYNJ1XXBcepaY61RE4on2hlQRZjM9kfY&cc=jfTizTSiuMmgWrefnWmJK1bH4Tmqa_7qxmxgWR_OH6k

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk5StwJAmvoOpF9sbxtDPGSfzO_C82cBN4D3DWlRK4SPjGmfY7z9ZWg

Successfully saved authorization token.


###### **1.2 Instalación de bibliotecas**
Como en cuadernos anteriores, por cada sesión que se inicia es necesario instalar aquellas librerías que sean necesarios para los fines que se desean alcanzar.

In [3]:
# Permite cargar shapefiles y convertirlos en una featureCollectionLoad 
!pip install geetools
!pip install fastkml
!pip install pycrs
!pip install geemap

# Permite mostrar series como las vemos en Earth Engine Code Editor 
!pip install ipygee

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 KB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 5.8 MB/s eta 0:00:00
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=b57d70e62425ee25e047c19a69796e948bbae5b27e8aebc5e9f50c57219459f0
  Stored in directory: /root/.cache/pip/wheels/20/4c/bf/0228d788d820ac3cf05cbaa753965eb6396b5d3fe11ed912e5
Successfully built geetools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pygeoif: filename=pygeoif-0.7-py3-none-any.whl size=19015 sha256=5b36475aa074942706de743a36ca960023057e817a7e1ebf286f8e7594265577
  Stored in directory: /root/.cache/pip/

#### **2. Exportación de imágenes**

Para exportar una imagen a nuestra unidad en Googel Drive se usa la función **ee.batch.Export.image.toDrive**, indicando como parámetros:
* `image`: La imagen que se desea exportar.
* `description`: Nombre a asignar a la imagen exportada.
* `folder`: Carpeta dentro de nuestra unidad donde se desea almacenar la imagen.
* `scale`: resolución espacial de la imagen a exportar.
* `region`: Área geográfica a exportar dentro de la imagen.

En el ejemplo presentado se ha extraido el rectángulo mínimo envolvente de las *eeFeatures* de la *eeFeatureCollection* empleadno para ello el metodo **bounds()**.

Llamado a la función `ee.batch.Export.image.toDrive` lo que se esta haciendo es crear una tarea que posteriormente es necesario lanzarla mediante el método **start()**.

En este script se exporta una imagen en formato TIFF con cuatro bandas.

En GEE no es posible exportar directamente una colección de imágenes. Para solucionarlo, tal y como vimos en cuadernos anteriores, se puede hacer un bucle que recorra una lista con el nombre (identificador) de las imagenes de una colección. Con este nombre, y dentro del bucle, se crea la variable de tipo *image* y se genera la tarea del exportar a imagen para finalmente lanzarla a ejecución antes de ir pasar al sigueinte elemento de la lista.

In [4]:
import os
import geemap
from geetools import batch

#os.chdir('/content/drive/My Drive/')

#Leemos un shapeFile alojado en nuetro espacio en Google Drive
path_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/HIBA/Cuadernos Colab/5 - Exportando resultados/datos/area.shp')
tabla = geemap.shp_to_ee(path_file)
parcela = ee.FeatureCollection(tabla)

# Cargamos una escena Sentinel 2.
sentinel = ee.Image('COPERNICUS/S2/20170412T110621_20170412T111708_T30SUG')\
            .select(['B4', 'B3', 'B2','B8'])

bouding_box=parcela.geometry().bounds()

# Exportamos la imagen, indicando escala y el ámbito.
task = ee.batch.Export.image.toDrive(**{
    'image': sentinel,
    'description': '20210306_Ejemplo_exportacion',
    'folder':'',
    'scale': 10,
    'region': bouding_box.getInfo()['coordinates']
})
task.start()

task = ee.batch.Export.image.toDrive(**{
    'image': sentinel,
    'description': '20210306_Ejemplo_exportacion_100',
    'folder':'',
    'scale': 100,
    'region': bouding_box.getInfo()['coordinates']
})
task.start()

#### **3 Exportación de videos**

Siguiendo la misma lógica que en el caso anterior es posible exportar un video a modo de time lapse empleando las imagenes de una colección de imágenes, empleando la función **ee.batch.Export.video.toDrive**. Los parámetros de esta función son:

* `collection`: Variable con la colección de imágenes a emplear para generar el video.
* `description`: Nombre a asignar a la imagen exportada.
* `dimesions`: El tamaño del video.
* `framesPerSecond`: Determinará la duración del video.
* `region`: Área geográfica a exportar dentro de la image.

Con la función se genera una tarea que finalmente habrá que ejecutarla mediante la llanada a **start()**.



In [5]:
import os
import time
import math
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as dt
from datetime import datetime as dateTim

from geetools import batch

from ipygee import *
import json

os.chdir('/content/drive/My Drive/')

#Leemos un shapeFile alojado en nuetro espacio en Google Drive
path_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/HIBA/Cuadernos Colab/5 - Exportando resultados/datos/area.shp')
tabla = geemap.shp_to_ee(path_file)
parcela = ee.FeatureCollection(tabla)

#Definimos las fechas de inicio y fin de la colección
start_date='2016-09-01'
end_date='2021-03-01'

#Creamos una colección de imágenes
coleccion = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(parcela)\
          .filterDate(start_date, end_date)\
          .filterMetadata('CLOUD_COVERAGE_ASSESSMENT' ,'less_than',5)\
          .filter(ee.Filter.eq('MGRS_TILE','30SUH'))\
          .select(['B8', 'B3', 'B4'])

print ("Numero de imágenes de la colección: ",coleccion.size().getInfo())


bouding_box=parcela.geometry().bounds()

def pasar8bits(entrada):
   salida= entrada.multiply(0.0256).uint8()
   return salida

coleccion=coleccion.map(pasar8bits)

tarea = ee.batch.Export.video.toDrive(**{
  'collection': coleccion,
  'description': '20210306_ejemplo_video',
  'dimensions': 720,
  'framesPerSecond': 5,
  'region': bouding_box.getInfo()['coordinates']
});

tarea.start()

while tarea.active():
  print('Polling for task (id: {}).'.format(tarea.id))
  time.sleep(5)


Numero de imágenes de la colección:  193
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKYLTRZC2QBV6CV7CHQXU).
Polling for task (id: NOCTKY